In [ ]:
library(foreign)
library(sp)
library(maptools)
library(spatstat)
library(ggmap)
library(rgdal)

In [ ]:
## Plot crime events
crime_points <- read.dbf("chicago_crimes_all_beats.dbf")
local_crime_points <- read.dbf("chicago_crimes_selected_beats.dbf")

In [ ]:
lon <- as.numeric(quantile(local_crime_points[, 'Longitude'], .55))
lat <- mean(local_crime_points[,'Latitude'])
chicago <- get_map(location = c(lon, lat), zoom = 14, source = "osm", color = "bw")

In [ ]:
plotted_points <- geom_point(data = crime_points, 
    aes(x = Longitude, y = Latitude), size = .8,
    shape = 20, color = "darkred", show.legend = FALSE)

In [ ]:
ggmap(chicago) + plotted_points +
    theme(plot.title = element_text(hjust = .5), 
    axis.title.y = element_blank(), axis.title.x = element_blank()) + 
    ggtitle(expression(atop(paste('Bloomingdale Trail'), atop(paste('Crime Locations Since 6/6/2015'), ""))))

In [ ]:
## Read data
data <- readShapeSpatial("chicago_crimes_selected_beats.shp")
chicago_beats_subset <- readShapeSpatial("chicago_beats_subset.shp")

In [ ]:
## Plot crime locations
plot(chicago_beats_subset, main = "Bloomingdale Trail: Adjacent Police Beats")
mtext("Crime Locations Since 6/6/2015")
plot(data_pt, pch = 16, cex = 0.3, col = "darkred", add = T)

In [ ]:
## Extract point data
data_pt <- as.ppp(data)
data_pt_only <- unmark(data_pt)

In [ ]:
## Get projected values
EPSG <- 32700 - round((45 + lat)/90)*100 + round((183 + lon)/6)
coords <- SpatialPoints(coords(data_pt_only), proj4string = CRS("+proj=longlat"))
proj_xy <- spTransform(coords, CRS(paste0("+init=epsg:", EPSG, "")))

In [ ]:
proj_data_pt <- unmark(as.ppp(proj_xy))
plot(proj_data_pt)
summary(proj_data_pt)

In [ ]:
## Set spatial domain window
chicago_beats_subset <- as.owin(chicago_beats_subset)
beat_data <- data_pt_only[chicago_beats_subset]
plot(chicago_beats_subset, main = "Bloomingdale Trail: Adjacent Police Beats")
plot(data_pt_only, pch = 16, cex = 0.4, col = "dark red", add = T)
mtext("Crime Locations Since 6/6/2015")

In [ ]:
qc <- quadratcount(beat_data, nx = 1, ny = 5)
plot(qc)

In [ ]:
plot(chicago_beats_subset)
plot(intensity(qc, image=T), add = T)
plot(data_pt_only, add = T)

In [ ]:
## Test for centrography
library(sp)
library(aspace)
library(maptools)
library(plotrix)

In [ ]:
## Get matrix
storexy <- coordinates(data)
store <- data.frame(storexy, row.names = seq(dim(data)[1]))

In [ ]:
## Calculate data tendencies
mnc <- mean_centre(id = 10, points = store)
mdc <- median_centre(id = 11, points = store)
cmind <- CF(id = 12, points = store)

In [ ]:
plot(store, main="Crime", cex = 0.5, asp=1)
points(mnc$CENTRE.x, mnc$CENTRE.y, cex=1.5, pch=3, col="red")
points(mdc$median.x, mdc$median.y, cex=1.5, pch=4, col="blue")
points(cmind$CF.x, cmind$CF.y, cex=1.5, pch=2, col="green")

In [ ]:
## Kernel Density
dd1 <- density(beat_data)
attr(dd1,"sigma")

In [ ]:
plot(chicago_beats_subset, main = "Bloomingdale Trail: Adjacent Police Beats")
mtext("Crime Locations Since 6/6/2015 - Kernel Density")
plot(dd1, add = T)
plot(beat_data, pch = 16, cex = 0.4, col = "black", add = T)

In [ ]:
## Minimum Squared Error
bw1 <- bw.diggle(beat_data)
bw1
plot(bw1)

In [ ]:
plot(chicago_beats_subset, main = "Bloomingdale Trail: Adjacent Police Beats")
mtext("Crime Locations Since 6/6/2015")
plot(density(beat_data, sigma = bw1), add = T)
plot(beat_data, pch = 16, cex = 0.4, col = "black", add = T)

In [ ]:
## Cumulative distance
bw3 <- bw.frac(beat_data)
bw3
plot(bw3)

In [ ]:
plot(chicago_beats_subset, main = "Bloomingdale Trail: Adjacent Police Beats")
mtext("Crime Locations Since 6/6/2015")
plot(density(beat_data, sigma = bw3), add = T)
plot(beat_data, pch = 16, cex = 0.4, col = "black", add = T)

In [ ]:
## Random tesselation
dd2 <- adaptive.density(beat_data, f = 0.1, nrep = 5)
plot(chicago_beats_subset, main = "Bloomingdale Trail: Adjacent Police Beats")
mtext("Crime Locations Since 6/6/2015")
plot(dd2, add = T)
plot(beat_data, pch = 16, cex = 0.4, col = "black", add = T)